In [33]:
import numpy as np
import pandas as pd
#import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler


import tensorflow.compat.v1 as tf

In [34]:
rating = pd.read_csv('FoodRatings2.csv', sep=';', on_bad_lines='skip', encoding="latin-1")
food_rating = rating.copy()
cols = ['UserID', 'FoodID', 'Rating']

In [35]:
rating_count = (food_rating.
     groupby(by = ['foodid'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_Food'})
     [['foodid', 'RatingCount_Food']])
rating_count

,foodid,RatingCount_Food
0,1,40
1,2,86
2,3,74
3,4,76
4,5,80
...,...,...
85,86,95
86,87,79
87,88,91
88,89,88


In [36]:
threshold = 10
rating_count = rating_count.query('RatingCount_Food >= @threshold')
rating_count.head()

,foodid,RatingCount_Food
0,1,40
1,2,86
2,3,74
3,4,76
4,5,80


In [37]:
user_rating = pd.merge(rating_count, food_rating, left_on='foodid', right_on='foodid', how='left')

In [50]:
user_count = (user_rating.
 groupby(by = ['userid'])['rating'].
 count().
 reset_index().
 rename(columns = {'rating': 'RatingCount_user'})
 [['userid', 'RatingCount_user']])
user_count

,userid,RatingCount_user
0,1,33
1,2,86
2,3,69
3,4,69
4,5,69
...,...,...
96,97,79
97,98,86
98,99,89
99,100,28


In [39]:
combined = user_rating.merge(user_count, left_on = 'userid', right_on = 'userid', how = 'inner')

In [40]:
scaler = MinMaxScaler()
combined['rating'] = combined['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['rating'].values.reshape(-1,1)))
combined['rating'] = rating_scaled
combined['rating']

0       0.75
1       0.00
2       1.00
3       1.00
4       0.25
        ... 
7493    0.25
7494    0.75
7495    0.75
7496    0.75
7497    0.75
Name: rating, Length: 7498, dtype: float64

In [41]:
combined = combined.drop_duplicates(['userid', 'foodid'])
user_food_matrix = combined.pivot(index='userid', columns='foodid', values='rating')
user_food_matrix.fillna(0, inplace=True)

users = user_food_matrix.index.tolist()
books = user_food_matrix.columns.tolist()

user_food_matrix = user_food_matrix.values

In [42]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [43]:
num_input = combined['foodid'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

In [44]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

In [45]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X

In [46]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

In [47]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

In [48]:
#saver = tf.compat.v1.train.Saver()
with tf.Session() as session:
    epochs = 100
    batch_size = 35
    

    session.run(init)
    session.run(local_init)

    num_batches = int(user_food_matrix.shape[0] / batch_size)
    user_food_matrix = np.array_split(user_food_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_food_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))
    
    # Save the final model
    #saver.save(session, 'model.ckpt')

    user_food_matrix = np.concatenate(user_food_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_food_matrix})

    pred_data = pd.concat([pred_data, pd.DataFrame(preds)])

    pred_data = pred_data.stack().reset_index(name='rating')
    pred_data.columns = ['userid', 'foodid', 'rating']
    pred_data['userid'] = pred_data['userid'].map(lambda value: users[value])
    pred_data['foodid'] = pred_data['foodid'].map(lambda value: books[value])
    
    keys = ['userid', 'foodid']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['userid', 'rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('userid').head(10)
    
        


epoch: 1 Loss: 0.2793520539999008
epoch: 2 Loss: 0.27931858599185944
epoch: 3 Loss: 0.279281347990036
epoch: 4 Loss: 0.27924004197120667
epoch: 5 Loss: 0.2791941165924072
epoch: 6 Loss: 0.2791431099176407
epoch: 7 Loss: 0.27908647060394287
epoch: 8 Loss: 0.27902354300022125
epoch: 9 Loss: 0.2789537161588669
epoch: 10 Loss: 0.2788761258125305
epoch: 11 Loss: 0.2787899821996689
epoch: 12 Loss: 0.278694286942482
epoch: 13 Loss: 0.27858807146549225
epoch: 14 Loss: 0.2784701734781265
epoch: 15 Loss: 0.2783392667770386
epoch: 16 Loss: 0.27819399535655975
epoch: 17 Loss: 0.27803273499011993
epoch: 18 Loss: 0.2778538316488266
epoch: 19 Loss: 0.27765530347824097
epoch: 20 Loss: 0.2774350941181183
epoch: 21 Loss: 0.2771908789873123
epoch: 22 Loss: 0.27692006528377533
epoch: 23 Loss: 0.2766198366880417
epoch: 24 Loss: 0.27628709375858307
epoch: 25 Loss: 0.27591848373413086
epoch: 26 Loss: 0.2755102217197418
epoch: 27 Loss: 0.27505825459957123
epoch: 28 Loss: 0.2745581120252609
epoch: 29 Loss: 0.2

In [49]:
top_ten_ranked.loc[top_ten_ranked['userid'] == 123]

,userid,foodid,rating
9038,123,39,0.436646
9066,123,67,0.424406
9059,123,60,0.416024
9050,123,51,0.415427
9071,123,72,0.414430
9051,123,52,0.402113
9009,123,10,0.393258
9005,123,6,0.391373
9010,123,11,0.378176
9060,123,61,0.375126
